In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import json

# ============================================================================
# 1. FOREX FACTORY CALENDAR API (Web Scraping - No official API)
# ============================================================================

def fetch_forexfactory_news():
    """
    Fetch economic calendar from Forex Factory
    Note: Forex Factory doesn't have an official API, so we scrape the website
    """
    print("\n" + "="*100)
    print("FOREX FACTORY ECONOMIC CALENDAR")
    print("="*100)
    
    try:
        url = "https://www.forexfactory.com/calendar"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find calendar table
        calendar_rows = soup.find_all('tr', class_='calendar__row')
        
        events = []
        current_date = None
        
        for row in calendar_rows:
            # Get date if available
            date_cell = row.find('td', class_='calendar__cell calendar__date')
            if date_cell and date_cell.text.strip():
                current_date = date_cell.text.strip()
            
            # Get time
            time_cell = row.find('td', class_='calendar__cell calendar__time')
            time_str = time_cell.text.strip() if time_cell else ''
            
            # Get currency
            currency_cell = row.find('td', class_='calendar__cell calendar__currency')
            currency = currency_cell.text.strip() if currency_cell else ''
            
            # Get impact (importance)
            impact_cell = row.find('td', class_='calendar__cell calendar__impact')
            impact_span = impact_cell.find('span') if impact_cell else None
            impact = ''
            if impact_span:
                if 'icon--ff-impact-red' in impact_span.get('class', []):
                    impact = 'High'
                elif 'icon--ff-impact-ora' in impact_span.get('class', []):
                    impact = 'Medium'
                elif 'icon--ff-impact-yel' in impact_span.get('class', []):
                    impact = 'Low'
            
            # Get event name
            event_cell = row.find('td', class_='calendar__cell calendar__event')
            event_name = event_cell.text.strip() if event_cell else ''
            
            # Get actual, forecast, previous values
            actual_cell = row.find('td', class_='calendar__cell calendar__actual')
            actual = actual_cell.text.strip() if actual_cell else ''
            
            forecast_cell = row.find('td', class_='calendar__cell calendar__forecast')
            forecast = forecast_cell.text.strip() if forecast_cell else ''
            
            previous_cell = row.find('td', class_='calendar__cell calendar__previous')
            previous = previous_cell.text.strip() if previous_cell else ''
            
            if event_name:
                events.append({
                    'Date': current_date,
                    'Time': time_str,
                    'Currency': currency,
                    'Impact': impact,
                    'Event': event_name,
                    'Actual': actual,
                    'Forecast': forecast,
                    'Previous': previous
                })
        
        if events:
            df = pd.DataFrame(events)
            print(f"\nFetched {len(events)} events from Forex Factory")
            print(df.to_string(index=False))
            return df
        else:
            print("No events found")
            return None
            
    except Exception as e:
        print(f"Error fetching Forex Factory data: {e}")
        return None

# ============================================================================
# 2. INVESTING.COM ECONOMIC CALENDAR (Web Scraping)
# ============================================================================

def fetch_investing_com_news():
    """
    Fetch economic calendar from Investing.com
    Note: Requires web scraping as API access is restricted
    """
    print("\n" + "="*100)
    print("INVESTING.COM ECONOMIC CALENDAR")
    print("="*100)
    
    try:
        url = "https://www.investing.com/economic-calendar/"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find economic calendar events
        events = []
        
        # Look for calendar table/rows (structure may vary)
        table = soup.find('table', {'id': 'economicCalendarData'})
        
        if table:
            rows = table.find_all('tr', class_='js-event-item')
            
            for row in rows:
                time_cell = row.find('td', class_='time')
                currency_cell = row.find('td', class_='flagCur')
                event_cell = row.find('td', class_='event')
                actual_cell = row.find('td', {'id': lambda x: x and 'eventActual' in x})
                forecast_cell = row.find('td', {'id': lambda x: x and 'eventForecast' in x})
                previous_cell = row.find('td', {'id': lambda x: x and 'eventPrevious' in x})
                
                # Get importance (bull icons)
                importance_cell = row.find('td', class_='sentiment')
                importance = 'Low'
                if importance_cell:
                    bulls = len(importance_cell.find_all('i', class_='grayFullBullishIcon'))
                    if bulls == 3:
                        importance = 'High'
                    elif bulls == 2:
                        importance = 'Medium'
                
                events.append({
                    'Time': time_cell.text.strip() if time_cell else '',
                    'Currency': currency_cell.text.strip() if currency_cell else '',
                    'Importance': importance,
                    'Event': event_cell.text.strip() if event_cell else '',
                    'Actual': actual_cell.text.strip() if actual_cell else '',
                    'Forecast': forecast_cell.text.strip() if forecast_cell else '',
                    'Previous': previous_cell.text.strip() if previous_cell else ''
                })
        
        if events:
            df = pd.DataFrame(events)
            print(f"\nFetched {len(events)} events from Investing.com")
            print(df.to_string(index=False))
            return df
        else:
            print("No events found or structure changed")
            return None
            
    except Exception as e:
        print(f"Error fetching Investing.com data: {e}")
        return None

# ============================================================================
# 3. TRADING ECONOMICS API (Requires API Key)
# ============================================================================

def fetch_trading_economics_news(api_key=None):
    """
    Fetch economic calendar from Trading Economics API
    Get free API key at: https://tradingeconomics.com/api/
    """
    print("\n" + "="*100)
    print("TRADING ECONOMICS API")
    print("="*100)
    
    if not api_key:
        print("\nAPI Key required!")
        print("Get your free API key at: https://tradingeconomics.com/api/")
        print("Then call: fetch_trading_economics_news(api_key='YOUR_KEY')")
        return None
    
    try:
        # Get today's calendar
        today = datetime.now().strftime('%Y-%m-%d')
        url = f"https://api.tradingeconomics.com/calendar/country/all/{today}"
        
        params = {
            'c': api_key,
            'f': 'json'
        }
        
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        if data:
            events = []
            for item in data:
                events.append({
                    'Date': item.get('Date'),
                    'Country': item.get('Country'),
                    'Category': item.get('Category'),
                    'Event': item.get('Event'),
                    'Importance': item.get('Importance', 'N/A'),
                    'Actual': item.get('Actual'),
                    'Forecast': item.get('Forecast'),
                    'Previous': item.get('Previous'),
                    'Currency': item.get('Currency')
                })
            
            df = pd.DataFrame(events)
            print(f"\nFetched {len(events)} events from Trading Economics")
            print(df.to_string(index=False))
            return df
        else:
            print("No events found")
            return None
            
    except Exception as e:
        print(f"Error fetching Trading Economics data: {e}")
        return None

# ============================================================================
# 4. NEWSAPI.ORG (General News - Requires API Key)
# ============================================================================

def fetch_newsapi_org(api_key=None, query='forex OR economy OR trading', days_back=1):
    """
    Fetch general news from NewsAPI.org
    Get free API key at: https://newsapi.org/
    """
    print("\n" + "="*100)
    print("NEWSAPI.ORG - FINANCIAL NEWS")
    print("="*100)
    
    if not api_key:
        print("\nAPI Key required!")
        print("Get your free API key at: https://newsapi.org/")
        print("Then call: fetch_newsapi_org(api_key='YOUR_KEY')")
        return None
    
    try:
        # Calculate date range
        from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
        
        url = "https://newsapi.org/v2/everything"
        
        params = {
            'apiKey': api_key,
            'q': query,
            'from': from_date,
            'sortBy': 'publishedAt',
            'language': 'en',
            'pageSize': 100
        }
        
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        if data.get('status') == 'ok' and data.get('articles'):
            articles = data['articles']
            
            news_list = []
            for article in articles:
                news_list.append({
                    'Published': article.get('publishedAt'),
                    'Source': article.get('source', {}).get('name'),
                    'Title': article.get('title'),
                    'Description': article.get('description'),
                    'URL': article.get('url')
                })
            
            df = pd.DataFrame(news_list)
            print(f"\nFetched {len(news_list)} news articles")
            print(df[['Published', 'Source', 'Title']].to_string(index=False))
            return df
        else:
            print(f"No articles found or error: {data.get('message', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"Error fetching NewsAPI data: {e}")
        return None

# ============================================================================
# MAIN FUNCTION - FETCH FROM ALL SOURCES
# ============================================================================

def fetch_all_news(trading_economics_key=None, newsapi_key=None):
    """
    Fetch news from all available sources
    """
    print("\n" + "="*100)
    print("FETCHING NEWS FROM ALL SOURCES")
    print("="*100)
    
    results = {}
    
    # 1. Forex Factory (free, no API key needed)
    print("\n[1/4] Fetching from Forex Factory...")
    results['forexfactory'] = fetch_forexfactory_news()
    
    # 2. Investing.com (free, no API key needed)
    print("\n[2/4] Fetching from Investing.com...")
    results['investing'] = fetch_investing_com_news()
    
    # 3. Trading Economics (requires API key)
    print("\n[3/4] Fetching from Trading Economics...")
    results['trading_economics'] = fetch_trading_economics_news(trading_economics_key)
    
    # 4. NewsAPI.org (requires API key)
    print("\n[4/4] Fetching from NewsAPI.org...")
    results['newsapi'] = fetch_newsapi_org(newsapi_key)
    
    print("\n" + "="*100)
    print("SUMMARY")
    print("="*100)
    for source, data in results.items():
        if data is not None:
            print(f"✓ {source}: {len(data)} items fetched")
        else:
            print(f"✗ {source}: Failed or requires API key")
    
    return results

# ============================================================================
# USAGE EXAMPLES
# ============================================================================

if __name__ == "__main__":
    print("""
    ╔═══════════════════════════════════════════════════════════════════════╗
    ║                    MULTI-SOURCE NEWS FETCHER                          ║
    ║                    For Trading & Economic News                        ║
    ╚═══════════════════════════════════════════════════════════════════════╝
    
    REQUIRED PACKAGES:
    pip install requests beautifulsoup4 pandas
    
    API KEYS (Optional but recommended):
    1. Trading Economics: https://tradingeconomics.com/api/
    2. NewsAPI.org: https://newsapi.org/
    
    """)
    
    # Option 1: Fetch from all sources (free sources only)
    print("\n>>> Fetching from free sources (Forex Factory & Investing.com)...\n")
    results = fetch_all_news()
    
    # Option 2: Fetch with API keys (uncomment and add your keys)
    # results = fetch_all_news(
    #     trading_economics_key='YOUR_TRADING_ECONOMICS_KEY',
    #     newsapi_key='YOUR_NEWSAPI_KEY'
    # )
    
    # Option 3: Fetch from individual sources
    # df_ff = fetch_forexfactory_news()
    # df_inv = fetch_investing_com_news()
    # df_te = fetch_trading_economics_news('YOUR_KEY')
    # df_news = fetch_newsapi_org('YOUR_KEY')


    ╔═══════════════════════════════════════════════════════════════════════╗
    ║                    MULTI-SOURCE NEWS FETCHER                          ║
    ║                    For Trading & Economic News                        ║
    ╚═══════════════════════════════════════════════════════════════════════╝

    REQUIRED PACKAGES:
    pip install requests beautifulsoup4 pandas

    API KEYS (Optional but recommended):
    1. Trading Economics: https://tradingeconomics.com/api/
    2. NewsAPI.org: https://newsapi.org/

    

>>> Fetching from free sources (Forex Factory & Investing.com)...


FETCHING NEWS FROM ALL SOURCES

[1/4] Fetching from Forex Factory...

FOREX FACTORY ECONOMIC CALENDAR
Error fetching Forex Factory data: 403 Client Error: Forbidden for url: https://www.forexfactory.com/calendar

[2/4] Fetching from Investing.com...

INVESTING.COM ECONOMIC CALENDAR

Fetched 39 events from Investing.com
 Time Currency Importance                                         Event    Ac